In [2]:
import requests
import json
from typing import List, Dict, Any

class DatabaseTypeFieldUpdater:
    def __init__(self, api_base_url: str = "http://localhost:3000/api/contracts"):
        self.api_base_url = api_base_url
    
    def fetch_all_contracts(self) -> List[Dict[str, Any]]:
        """Fetch all contracts from the database"""
        try:
            response = requests.get(self.api_base_url)
            if response.status_code == 200:
                contracts = response.json()
                print(f"✓ Fetched {len(contracts)} contracts from database")
                return contracts
            else:
                print(f"✗ Error fetching contracts: {response.status_code}")
                return []
        except Exception as e:
            print(f"✗ Error fetching contracts: {e}")
            return []
    
    def determine_type_from_name(self, contract_name: str) -> str:
        """Determine the Type based on contract name"""
        if "(LGC)" in contract_name:
            return "Green"
        else:
            return "Energy"
    
    def update_contract_with_type(self, contract: Dict[str, Any]) -> bool:
        """Update a single contract to add the Type field"""
        try:
            contract_id = contract.get('_id')
            if not contract_id:
                print(f"✗ No ID found for contract: {contract.get('name', 'Unknown')}")
                return False
            
            # Determine the Type based on contract name
            contract_name = contract.get('name', '')
            contract_type = self.determine_type_from_name(contract_name)
            
            # Prepare update payload - only add Type field, keep existing unit
            update_payload = {
                'id': contract_id,
                'contractType': contract_type  # Add the new Type field
            }
            
            # Make API call to update contract
            response = requests.put(self.api_base_url, json=update_payload)
            
            if response.status_code == 200:
                print(f"✓ Updated contract: {contract_name} → Type: {contract_type}")
                return True
            else:
                print(f"✗ Error updating contract {contract_name}: {response.status_code} - {response.text}")
                return False
                
        except Exception as e:
            print(f"✗ Error updating contract {contract.get('name', 'Unknown')}: {e}")
            return False
    
    def preview_changes(self, contracts: List[Dict[str, Any]]) -> None:
        """Preview what changes will be made"""
        print("\n" + "="*80)
        print("PREVIEW OF CHANGES")
        print("="*80)
        
        energy_count = 0
        green_count = 0
        
        for contract in contracts:
            name = contract.get('name', 'Unknown')
            current_unit = contract.get('unit', 'N/A')
            new_type = self.determine_type_from_name(name)
            
            if new_type == "Green":
                green_count += 1
                print(f"🟢 {name}")
                print(f"   Current unit: {current_unit} → Will add Type: Green")
            else:
                energy_count += 1
                print(f"🔵 {name}")
                print(f"   Current unit: {current_unit} → Will add Type: Energy")
        
        print(f"\nSummary:")
        print(f"  Contracts to be set to Type 'Energy': {energy_count}")
        print(f"  Contracts to be set to Type 'Green': {green_count}")
        print(f"  Total contracts: {len(contracts)}")
        print(f"  Note: Existing 'unit' field will be preserved")
    
    def update_all_contracts(self, dry_run: bool = True) -> None:
        """Main method to update all contracts with Type field"""
        print("Starting database Type field addition process...")
        print(f"Mode: {'DRY RUN (preview only)' if dry_run else 'LIVE UPDATE'}")
        
        # Fetch all contracts
        contracts = self.fetch_all_contracts()
        if not contracts:
            print("No contracts found. Exiting.")
            return
        
        # Preview changes
        self.preview_changes(contracts)
        
        if dry_run:
            print(f"\n{'='*80}")
            print("DRY RUN COMPLETE")
            print("="*80)
            print("This was a preview. To execute the updates, run with dry_run=False")
            return
        
        # Confirm before proceeding with live updates
        print(f"\n{'='*80}")
        print("READY TO EXECUTE LIVE UPDATES")
        print("="*80)
        confirm = input("Are you sure you want to proceed? (y/N): ").strip().lower()
        
        if confirm != 'y':
            print("Update cancelled.")
            return
        
        # Execute updates
        print("\nExecuting updates...")
        success_count = 0
        error_count = 0
        
        for contract in contracts:
            if self.update_contract_with_type(contract):
                success_count += 1
            else:
                error_count += 1
        
        # Final summary
        print(f"\n{'='*80}")
        print("UPDATE SUMMARY")
        print("="*80)
        print(f"Successfully updated: {success_count}")
        print(f"Errors: {error_count}")
        print(f"Total processed: {len(contracts)}")
        
        if error_count == 0:
            print("🎉 All contracts updated successfully!")
        else:
            print(f"⚠️  {error_count} contracts had errors during update")

# Example usage
if __name__ == "__main__":
    updater = DatabaseTypeFieldUpdater("http://localhost:3000/api/contracts")
    
    # First run a dry run to preview changes
    print("=== DRY RUN: Preview Changes ===")
    updater.update_all_contracts(dry_run=False)
    
    # Uncomment the line below to execute the actual updates
    # print("\n=== LIVE UPDATE: Execute Changes ===")
    # updater.update_all_contracts(dry_run=False)

=== DRY RUN: Preview Changes ===
Starting database Type field addition process...
Mode: LIVE UPDATE
✓ Fetched 354 contracts from database

PREVIEW OF CHANGES
🔵 BVC TOU
   Current unit: Energy → Will add Type: Energy
🟢 BVC TOU (LGC)
   Current unit: Green → Will add Type: Green
🔵 CSIRO TOU
   Current unit: MWh → Will add Type: Energy
🟢 CSIRO TOU (LGC)
   Current unit: MWh → Will add Type: Green
🔵 Fairfield Non-TOU
   Current unit: MWh → Will add Type: Energy
🟢 Fairfield Non-TOU (LGC)
   Current unit: MWh → Will add Type: Green
🔵 Fairfield TOU
   Current unit: MWh → Will add Type: Energy
🟢 Fairfield TOU (LGC)
   Current unit: MWh → Will add Type: Green
🔵 SBS TOU
   Current unit: MWh → Will add Type: Energy
🟢 SBS TOU (LGC)
   Current unit: Green → Will add Type: Green
🔵 SSROC Non-TOU
   Current unit: MWh → Will add Type: Energy
🟢 SSROC Non-TOU (LGC)
   Current unit: MWh → Will add Type: Green
🔵 SSROC TOU
   Current unit: MWh → Will add Type: Energy
🟢 SSROC TOU (LGC)
   Current unit: MWh →